In [ ]:
%load_ext tensorboard

In [ ]:
import tensorflow as tf
import datetime
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from keras import layers, Sequential, models
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [ ]:
class Autoencoder(Model):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = Sequential([
            layers.Dense(115, activation="relu"),
            layers.Dense(86, activation="relu"),
            layers.Dense(57, activation="relu"),
            layers.Dense(37, activation="relu"),
            layers.Dense(28, activation="relu")
        ])
        self.decoder = Sequential([
            layers.Dense(37, activation="relu"),
            layers.Dense(57, activation="relu"),
            layers.Dense(86, activation="relu"),
            layers.Dense(115, activation="sigmoid")
        ])
    
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
scaler = MinMaxScaler()

for i in range(9):
    # create a training set using the benign dataset
    benign = np.loadtxt(f'dataset/{i+1}.benign.csv', delimiter=",", skiprows=1)
    X_train = benign[:40000]
    # run 10 times to get an average
    for ii in range(10):
        x = scaler.fit_transform(X_train)
        
        # create an autoencoder model
        ae = Autoencoder()
        ae.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
        monitor = EarlyStopping(
            monitor='val_loss',
            min_delta=1e-9,
            patience=5,
            verbose=1,
            mode='auto'
        )
        
        log_dir = f"logs/fit/deep/device{i+1}/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        
        ae.fit(
            x=x,
            y=x,
            epochs=800,
            validation_split=0.3,
            shuffle=True,
            callbacks=[monitor, tensorboard_callback]
        )

In [ ]:
scaler = MinMaxScaler()

def train(ae, stage):
    for iii in range(3):
        # create a training set using the benign dataset
        benign = np.loadtxt(f'dataset/{iii+1}.benign.csv', delimiter=",", skiprows=1)
        X_train = benign[:40000]
        # run 10 times to get an average
        x = scaler.fit_transform(X_train)
            
        # create an autoencoder model
        ae.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
        monitor = EarlyStopping(
            monitor='val_loss',
            min_delta=1e-9,
            patience=5,
            verbose=1,
            mode='auto'
        )
        
        log_dir = f"logs/fit/{stage}/device{i+1}/device{iii+1}/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        
        ae.fit(
            x=x,
            y=x,
            epochs=800,
            validation_split=0.3,
            shuffle=True,
            callbacks=[monitor, tensorboard_callback]
        )

def tlencoder(ae):
    if ii == 0 and i == 0:
        ae.get_layer('sequential').get_layer('dense').trainable = False
        ae.get_layer('sequential').get_layer('dense_1').trainable = False
        ae.get_layer('sequential').get_layer('dense_2').trainable = False
    else:
        ae.get_layer(f'sequential_{(i*20)+(2*ii)}').get_layer(f'dense_{(90*i)+(9*ii)}').trainable = False
        ae.get_layer(f'sequential_{(i*20)+(2*ii)}').get_layer(f'dense_{(90*i)+(9*ii)+1}').trainable = False
        ae.get_layer(f'sequential_{(i*20)+(2*ii)}').get_layer(f'dense_{(90*i)+(9*ii)+2}').trainable = False
    train(ae, "encodertl")

def tlbottleneck(ae):
    if ii == 0 and i == 0:
        ae.get_layer('sequential').get_layer('dense_3').trainable = False
        ae.get_layer('sequential').get_layer('dense_4').trainable = False
    else:
        ae.get_layer(f'sequential_{(i*20)+(2*ii)}').get_layer(f'dense_{(90*i)+(9*ii)+3}').trainable = False
        ae.get_layer(f'sequential_{(i*20)+(2*ii)}').get_layer(f'dense_{(90*i)+(9*ii)+4}').trainable = False
        ae.get_layer(f'sequential_{(i*20)+(2*ii)+1}').get_layer(f'dense_{(90*i)+(9*ii)+5}').trainable = False
    train(ae, "bottlenecktl")
    
def tldecoder(ae):
    ae.get_layer(f'sequential_{(i*20)+(2*ii)+1}').get_layer(f'dense_{(90*i)+(9*ii)+6}').trainable = False
    ae.get_layer(f'sequential_{(i*20)+(2*ii)+1}').get_layer(f'dense_{(90*i)+(9*ii)+7}').trainable = False
    ae.get_layer(f'sequential_{(i*20)+(2*ii)+1}').get_layer(f'dense_{(90*i)+(9*ii)+8}').trainable = False
    train(ae, "decodertl")

for i in range(3):
    for ii in range(5):
        ae = models.load_model(f"neuralnetworks/deep/device{i+1}/run{ii+1}")
        tlencoder(ae)
        ae = models.load_model(f"neuralnetworks/deep/device{i+1}/run{ii+1}")
        tlbottleneck(ae)
        ae = models.load_model(f"neuralnetworks/deep/device{i+1}/run{ii+1}")
        tldecoder(ae)


In [ ]:
%tensorboard --logdir logs/fit/device2